# NB.
Сначала все ячейки до первой модели, потом начиная с обязательности полей и по "one by one", а потом возврат к первой модели.

In [39]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold

USD = 24.5
EUR =  26.9

DETAILS = [
    'SqrTotal', 
    'NFloors',
#    'NRooms', 
#    'Floor',
#    'SqrLive', 
#    'SqrKitchen'
]

In [2]:
ADMIN_REGION = [
# '1000001',
 '1002241',
 '1002242',
 '1002243',
 '1002246',
 '1002247',
 '1002248',
 '1002249',
 '1002250',
 '1002253',
 '1003198',
 ]

In [3]:
MASS_REGION = ['1000000',
 '1000002',
 '1000003',
 '1000005',
 '1000007',
 '1000008',
 '1000009',
 '1000010',
 '1000011',
 '1000012',
 '1000013',
 '1000014',
 '1000015',
 '1000016',
 '1000017',
 '1002254',
 '1002255',
 '1002256',
 '1002307',
 '1002308',
 '1002379',
 '1002380',
 '1002381',
 '1002382',
 '1002396',
 '1002478',
 '1002497',
 '1002537',
 '1002558',
 '1002559',
 '1002773',
 '1002852',
 '1002853',
 '1002854',
 '1002855',
 '1003012',
 '1003023',
 '1003091',
 '1003092',
 '1003094',
 '1003199',
 '1003200',
 '1003201',
 '1003202',
 '1003203',
 '1003326',
 '1003327',
 '1003437',
 '1003438',
 '1003439',
 '1003441',
 '1003503',
 '1003504',
 '1003505',
 '1003506',
 '1003705',
 '1003706',
 '1003708',
 '1003709',
 '1003711',
 '1003712',
 '1003713',
 '1003737',
 '1003748',
 '1003753',
 '1003754',
 '201326594',
 '201326596',
 '201326597',
 '201326598',
 '201326599',
 '201326600',
 '201326601',
 '201326602',
 '201326603',
 '201326604',
 '201326605',
 '201326606',
 '201326607',
 '201326608',
 '201326610',
 '201326611',
 '201326612',
 '201326613',
 '201326614',
 '201326615',
 '201326616',
 '201326617',
 '201326618',
 '201326619',
 '201326621',
 '385875969',
 '385875970',
 '385875971',
 '385876140',
 '385876141',
 '385876142',
 '385876143',
 '385876144',
 '385876145',
 '385876146',
 '385876147',
 '385876148',
 '385876149',
 '385876158',
 '385876159',
 '419430401',
 '419430402']

In [4]:
BIULD_TYPE = [
 '5039',
 '5040',
 '5041',
 '5042',
 '5044',
 '5045',
 '5048',
 '5053',
 '5055',
 '5061',
 '5063',
 '5064',
 '5065',
 '5066',
 '5067',
 '5069',
 '5070',
 '20451',
 '20452',
 '20485',
 '20492',
 '21486',
 '21737',
 '21777',
 '21778',
 '16797715',
 '16797718',
 '201326911',
 '201326915',
 '201326916',
 '201326917',
 '201326918',
 '201326919',
 '201326920',
 '385876155']

In [5]:
ROOMS_TYPE = ['213', '10200', '10201', '10202']
CEIL_KIND = ['5109', '5110', '5111']
BIULD_KIND = [
 '5094',
 '5095',
 '5096',
 '5097',
 '5098',
 '5099',
 '5100',
 '5101',
 '20528',
 '20629',
 '20647',
 '20812',
 '20813',
 '21594',
 '16877220',
 '385875982',
 '385875985']

In [6]:
NOMINAL_DETAILS = [
    ('RegionCode', ADMIN_REGION),
#    ('Massive', MASS_REGION),
#    ('BldType', BIULD_TYPE),
#    ('TypeRooms', ROOMS_TYPE),
#    ('CeilKind', CEIL_KIND),
#    ('BldKind', BIULD_KIND)
                  ]

In [7]:
BOOL_DETAILS = [
#    'Parking', 'Garage'
]

In [56]:
FEATURES = DETAILS + ["first/last floor"] +sum([i[1] for i in NOMINAL_DETAILS],[]) + BOOL_DETAILS

In [9]:
def nominal_vector(name, VALUES):
    vec = [0]*len(VALUES)
    try:
        vec[VALUES.index(name)] = 1
    except:
        pass
    return vec

In [63]:
def extract_data(flat):
    X = []
#    X += [float(flat['SqrTotal'])/int(flat['NRooms'])]
    for d in DETAILS:
        try:
            X += [float(flat[d])]
        except:
            X += [0]
#    if flat['Floor']=='1' or flat['Floor']==flat['NFloors']:
#        X+=[1]
#    else:
#        X+=[0]
    for n in NOMINAL_DETAILS:
        try:
            X += nominal_vector(flat[n[0]],n[1])
        except KeyError:
            X += [0] * len(n[1])
    for b in BOOL_DETAILS:
        try:
            X += [int(flat[b])]
        except:
            X += [0]    
    y=0
    try:
        y = float(flat['Price'])
    except:
        print "price problem"
        print flat
    try:
        if flat['PriceCurrencyID'] == '4':
            y *= USD
        if flat['PriceCurrencyID'] == '5':
            y *= EUR
    except:
        pass
    return [X, y]

In [11]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = []):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = 0.18, random_state = 3)
    regr = linear_model.LinearRegression(normalize=False, fit_intercept=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [12]:
#remove data without price
#field - str, data - list of dicts
def remove_with_no(field, t_data):
    c = 1
    while c!=0:
        c=0
        for d in t_data:
            try:
                d[field]
            except:
                #print d
                c+=1
                t_data.remove(d)
        print c
    return t_data

In [13]:
def get_X_y_from(t_data):
    t_cleaned_data = [extract_data(i) for i in t_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

## Load data & clean data

In [14]:
tree = ET.parse('./blagovest/Objects.xml')
root = tree.getroot()

In [44]:
blagovest_data_1 = root.findall('./Object/[@TownCode="300000"][@Transaction="Rent"][@ObjectType="Apartment"]')
blagovest_data_2 = root.findall('./Object/[@TownCode="300000"][@Transaction="Sale"][@ObjectType="Apartment"]')
blagovest_data_1 = [b.attrib for b in blagovest_data_1]
blagovest_data_2 = [b.attrib for b in blagovest_data_2]

In [ ]:
x=[[]]
for r_code in ADMIN_REGION:
    counts = {'1': 0, '2': 0, '3': 0, '4+': 0}
    ads = [i for i in blagovest_data_2 if i['RegionCode']==r_code]
    for child in ads:
        try:
            counts[child['NRooms']]+=1
            x[-1]+=[int(child['NRooms'])]
        except:
            counts['4+']+=1
            x[-1]+=[4]
    print "%s \t %d %d %d %d" %(r_code, counts['1'],counts['2'],counts['3'],counts['4+'])

In [45]:
blagovest_data_1 = remove_with_no('Price', blagovest_data_1)
blagovest_data_1 = remove_with_no('SqrTotal', blagovest_data_1)
blagovest_data_1 = remove_with_no('NFloors', blagovest_data_1)
#blagovest_data_1 = remove_with_no('NRooms', blagovest_data_1)
#blagovest_data_1 = remove_with_no('Floor', blagovest_data_1)
#blagovest_data_1 = remove_with_no('SqrLive', blagovest_data_1)
#blagovest_data_1 = remove_with_no('SqrKitchen', blagovest_data_1)
blagovest_data_1 = remove_with_no('RegionCode', blagovest_data_1)
#blagovest_data_1 = remove_with_no('Massive', blagovest_data_1)
#blagovest_data_1 = remove_with_no('BldType', blagovest_data_1)
#blagovest_data_1 = remove_with_no('TypeRooms', blagovest_data_1)
#blagovest_data_1 = remove_with_no('CeilKind', blagovest_data_1)
#blagovest_data_1 = remove_with_no('BldKind', blagovest_data_1)
#blagovest_data_1 = remove_with_no('Parking', blagovest_data_1)
#blagovest_data_1 = remove_with_no('Garage', blagovest_data_1)

0
64
4
0
0
0


In [ ]:
blagovest_data_2 = remove_with_no('Price', blagovest_data_2)
blagovest_data_2 = remove_with_no('SqrTotal', blagovest_data_2)
blagovest_data_2 = remove_with_no('NFloors', blagovest_data_2)
blagovest_data_2 = remove_with_no('NRooms', blagovest_data_2)
blagovest_data_2 = remove_with_no('Floor', blagovest_data_2)
#blagovest_data_2 = remove_with_no('BldType', blagovest_data_2)
#blagovest_data_2 = remove_with_no('SqrLive', blagovest_data_2)
#blagovest_data_2 = remove_with_no('SqrKitchen', blagovest_data_2)
blagovest_data_2 = remove_with_no('RegionCode', blagovest_data_2)
#blagovest_data_2 = remove_with_no('Massive', blagovest_data_2)
#blagovest_data_2 = remove_with_no('BldType', blagovest_data_2)
#blagovest_data_2 = remove_with_no('TypeRooms', blagovest_data_2)
#blagovest_data_2 = remove_with_no('CeilKind', blagovest_data_2)
#blagovest_data_2 = remove_with_no('BldKind', blagovest_data_2)

## First models

In [64]:
blagovest_X_1, blagovest_y_1 = get_X_y_from(blagovest_data_1)

In [53]:
print sum([i[3] for i in blagovest_X_1])
print len(blagovest_X_1)

71.0
2154


In [66]:
i=34
print blagovest_data_1[i]['Floor']
print blagovest_data_1[i]['NFloors']
print blagovest_X_1[i], blagovest_y_1[i], regr.predict(blagovest_X_1[i])

4
4
[ 70.   4.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.] 17000.0 [ 15651.51997627]


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [65]:
regr = process_with(blagovest_X_1, blagovest_y_1, info=True)

Total: 2154, train: 1766, test: 388
Residual sum of squares: 18961739.97
Train absolute: 3689.60
Test absolute: 3636.90
Absolute to mean: 23.50%
Train variance score: 0.33
Test variance score: 0.37


In [ ]:
blagovest_X_2, blagovest_y_2 = get_X_y_from(blagovest_data_2)

In [ ]:
print blagovest_X_2[0], blagovest_y_2[0]

In [ ]:
regr = process_with(blagovest_X_2, blagovest_y_2, info=True)

In [57]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4.2f \t %s" %(i[0], i[1])

Features sorted by their score:
0.0
13
101.00 	 SqrTotal
39.67 	 NFloors
-377.46 	 first/last floor
5303.74 	 1002241
5404.67 	 1002242
6170.05 	 1002243
7451.15 	 1002246
7882.43 	 1002247
8533.65 	 1002248
11329.05 	 1002249
8339.75 	 1002250
10384.94 	 1002253
5256.99 	 1003198


In [50]:
x=[[]]
for r_code in ADMIN_REGION:
    counts = {'1': 0, '2': 0, '3': 0, '4+': 0}
    ads = [i for i in blagovest_data_1 if i['RegionCode']==r_code]
    for child in ads:
        try:
            counts[child['NRooms']]+=1
        except:
            counts['4+']+=1
    print "%s \t %2.f %2.f %2.f %2.f %2.f" %(r_code, 
                                100*sum(counts.values())/float(len(blagovest_data_1)), 
                                counts['1']*100.0/sum(counts.values()),
                                counts['2']*100.0/sum(counts.values()),
                                counts['3']*100.0/sum(counts.values()),
                                counts['4+']*100.0/sum(counts.values()))

1002241 	  3 23 41 34  3
1002242 	 11 19 42 33  6
1002243 	  9 25 42 27  6
1002246 	 12 22 41 33  4
1002247 	  7 19 33 41  7
1002248 	 12 24 43 30  3
1002249 	 15 13 56 26  5
1002250 	  4 18 45 33  3
1002253 	 18 16 50 29  4
1003198 	  9 20 34 39  7


## Plots

In [67]:
plt.scatter([np.linalg.norm(i) for i in blagovest_X_1], blagovest_y_1, color='black')
plt.xticks(())
plt.yticks(())
plt.show()

## Making some fields requiered

In [ ]:
#remove_data_with_no_district_in_X
#[0:4], [4:15]
def make_field_required(t_data):
    ind = []
    X = get_X_y_from(t_data)[0]
    print "Elements: %d" % len(X)
    for i, x in enumerate(X):
        if np.count_nonzero(x[1:])==0:
            ind += [i]
    print "Deleted: %d" % len(ind)
    ind = sorted(ind, reverse=True)
    for i in ind:
        t=t_data.pop(i)
    return t_data

In [ ]:
blagovest_data_1=make_field_required(blagovest_data_1)

In [ ]:
metrovka_data_2=make_field_required(metrovka_data_2)

## Delete data with big diffs

In [19]:
def mean_diff(t_data, normalize = True, plots = True, info=True):
    X, y = get_X_y_from(t_data)
    print "Before:"
    before = process_with(X, y, return_short=True)
    print before
    
    if normalize:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
        y = min_max_scaler.fit_transform(y)
    
    pts = np.float32(zip(map(lambda x: np.linalg.norm(x), X),y))
    
    mean_pt = np.mean(pts,axis=0)
    mean_x,mean_y = mean_pt
    diffs = np.apply_along_axis(np.linalg.norm,1,pts - mean_pt)
    mean_diff = np.mean(diffs)

    filtered = np.float32([pts[i] for i,diff in enumerate(diffs) if diff <= 2*mean_diff])
    filtered_mean = np.mean(filtered,axis=0)
    
    t_data_f = [t_data[i] for i,diff in enumerate(diffs) if diff <= 2*mean_diff]
    X_f, y_f = get_X_y_from(t_data_f)
    print "After:"
    after = process_with(X_f,y_f, return_short=True)
    print after
    
#    plt.plot(pts[:,0],pts[:,1],'ro')
#    plt.plot(mean_x,mean_y,'w^')
#    plt.plot(filtered[:,0],filtered[:,1],'bo')
#    plt.plot(filtered_mean[0],filtered_mean[1],'w^')
#    plt.show()
    if after[1] > before[1] or after[0] < before[0]:
        return t_data_f
    return t_data

## Delete "outlaws"

In [20]:
def get_ind_by_y(X, y, ind_max = True):
    if ind_max:
        return np.argmax(y)
    else:
        return np.argmin(y)

In [21]:
def get_ind_by_x_norm(X, y, ind_max=True):
    norms = [np.linalg.norm(i) for i in X]
    #sorted_norm = [np.linalg.norm(i) for i in X]
    #sorted_norm.sort()
    if ind_max:
        #ind = norms.index(sorted_norm[-1])
        ind = norms.index(max(norms))
    else:
        #ind = norms.index(sorted_norm[1])
        ind = norms.index(min(norms))
    return ind

In [22]:
def get_ind_by_y_to_x_norm(X, y, ind_max=True):
    y_to_norm_x = [y[i]/np.linalg.norm(X[i]) for i in range(0,len(X))]
    if ind_max:
        return np.argmax(y_to_norm_x)
    else:
        return np.argmin(y_to_norm_x)

In [23]:
def cut_data_by(t_data, get_ind_cut_function, ind_max_flag=True):
    previous_sum = -100000 #10**32
    previous_score = -1000000 #-10**32
    X, y = get_X_y_from(t_data)
    [current_sum, current_score] = process_with(X, y, return_short=True)
    temp={}
    while current_score > previous_score or current_sum < previous_sum:
        previous_score = current_score
        previous_sum = current_sum

        inx = get_ind_cut_function(X, y,ind_max=ind_max_flag)
        #print t_data[norms.index(sorted_norm[-1])]['code'], sorted_norm[-1]
        temp = t_data.pop(inx)
        
        X, y = get_X_y_from(t_data)
        [current_sum, current_score] = process_with(X, y, return_short=True)
        
        print current_sum, previous_sum
        print current_score, previous_score
        print "-----"
    if temp != {}:
        t_data+=[temp]
    return t_data

## One by one

In [46]:
l = 1
while l!=len(blagovest_data_1):
    l = len(blagovest_data_1)
    print "by x norm max"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_x_norm, True)
    print "by y max"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_y, True)
    print "by y to x norm max"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_y_to_x_norm, True)
    print "by x norm min"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_x_norm, False)
    print "by y min"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_y, False)
    print "by y to x norm min"
    blagovest_data_1 = cut_data_by(blagovest_data_1, get_ind_by_y_to_x_norm, False)
    print "mean_diff"
    blagovest_data_1 = mean_diff(blagovest_data_1, normalize = True)

by x norm max
13808.9668882 13843.2648202
0.460160038813 0.526794524703
-----
13288.2586169 13808.9668882
0.56894758614 0.460160038813
-----
14034.2061261 13288.2586169
0.355357130512 0.56894758614
-----
by y max
14177.9686245 13675.3559585
0.382810264268 0.483090460915
-----
by y to x norm max
14177.9686245 13785.3241866
0.382810264268 0.502229223166
-----
by x norm min
14294.3476944 13785.3241866
0.383344549433 0.502229223166
-----
by y min
12865.8817405 13615.7849613
0.565623407514 0.509679055764
-----
13999.44375 12865.8817405
0.449737300609 0.565623407514
-----
by y to x norm min
13796.5954646 13306.9739895
0.441347318067 0.585545897364
-----
mean_diff
Before:
(13956.360775022491, 0.58255060374093215)
After:
(10293.497578782146, 0.55595285996870003)
by x norm max


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


10475.5699718 10293.4975788
0.496558173731 0.555952859969
-----
by y max
10832.4748444 10346.9832215
0.499433282114 0.505797224823
-----
by y to x norm max
10859.5675486 10870.0004532
0.498726507898 0.502052406598
-----
10414.6702046 10859.5675486
0.547469369542 0.498726507898
-----
9838.65478995 10414.6702046
0.578362594486 0.547469369542
-----
9947.72332245 9838.65478995
0.561028924125 0.578362594486
-----
by x norm min
9947.72332245 10473.7230457
0.561028924125 0.517389521459
-----
10046.4702196 9947.72332245
0.549606513549 0.561028924125
-----
by y min
10044.6086651 10184.5976633
0.549653469768 0.543585238532
-----
10148.4764561 10044.6086651
0.582677865997 0.549653469768
-----
10549.6923141 10148.4764561
0.524976069351 0.582677865997
-----
by y to x norm min
10557.2089722 11171.0771424
0.527093319481 0.521060160064
-----
10751.9092746 10557.2089722
0.518012348862 0.527093319481
-----
mean_diff
Before:
(11584.493904480692, 0.52938694005365527)
After:
(8123.5575857155382, 0.52960101

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


7865.09435081 8123.55758572
0.5599576167 0.529601019478
-----
8102.52335082 7865.09435081
0.544396471109 0.5599576167
-----
by y max
8082.82693467 8038.06841268
0.554522118711 0.536788912546
-----
8133.31956532 8082.82693467
0.564782863066 0.554522118711
-----
8114.11203295 8133.31956532
0.533908610694 0.564782863066
-----
7989.42584262 8114.11203295
0.561426198307 0.533908610694
-----
8135.55442023 7989.42584262
0.553949900809 0.561426198307
-----
by y to x norm max
8409.65857674 8197.28553242
0.558141459917 0.580042767105
-----
by x norm min
8409.60800666 8233.00234738
0.558233395686 0.574262462379
-----
by y min
8060.01352759 8237.42461404
0.595846193248 0.575772659937
-----
8235.66391816 8060.01352759
0.553819286159 0.595846193248
-----
by y to x norm min
8220.41822142 8213.53708712
0.549810227163 0.550374413387
-----
mean_diff
Before:
(8231.3807015094117, 0.54876648728082844)


...

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


3646.75942438 3586.36351497
0.388023179533 0.334801574894
-----
3741.37946559 3646.75942438
0.34061773947 0.388023179533
-----
by y max
3799.57435249 3653.65316404
0.334549670707 0.355104956789
-----
by y to x norm max
3800.47862833 3757.78153547
0.332589323834 0.358076765404
-----
by x norm min
3785.9548423 3757.78153547
0.337518900633 0.358076765404
-----
by y min
3772.77262093 3757.78153547
0.370989693191 0.358076765404
-----
3775.51495855 3772.77262093
0.320298949609 0.370989693191
-----
by y to x norm min
3801.39903134 3865.05424545
0.312593455298 0.311361638669
-----
3724.08426604 3801.39903134
0.313588206464 0.312593455298
-----
3825.76700153 3724.08426604
0.278935140355 0.313588206464
-----
mean_diff
Before:
(3816.5449789390209, 0.28399072018207117)
After:
(3474.8130518580929, 0.36356784793783181)
by x norm max
3779.76325302 3474.81305186
0.327898908805 0.363567847938
-----
by y max
3788.49877693 3485.90274485
0.329727400184 0.364576974652
-----
by y to x norm max


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


3783.94111148 3468.23794588
0.33294943078 0.366517051605
-----
by x norm min
3792.91082575 3489.52921951
0.326824748868 0.360288010677
-----
by y min
3712.70392529 3464.42052123
0.365171716978 0.372032478463
-----
by y to x norm min
3726.16541457 3665.38802343
0.364609500631 0.366344121283
-----
mean_diff
Before:
(3636.897840717073, 0.36907676075102991)
After:
(3710.1925079310631, 0.2919673089362701)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [ ]:
l = 1
while l!=len(blagovest_data_2):
    l = len(blagovest_data_2)
    print "by x norm max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_x_norm, True)
    print "by y max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y, True)
    print "by y to x norm max"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y_to_x_norm, True)
    print "by x norm min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_x_norm, False)
    print "by y min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y, False)
    print "by y to x norm min"
    blagovest_data_2 = cut_data_by(blagovest_data_2, get_ind_by_y_to_x_norm, False)
    print "mean_diff"
    blagovest_data_2 = mean_diff(blagovest_data_2, normalize = True)

## Saving and loading coefs

In [ ]:
coefs = map(lambda x: round(x, 4), regr.coef_)
f = zip(coefs, FEATURES)
print coefs

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'w') as outfile:
    pickle.dump(coefs, outfile)

In [ ]:
import pickle
with open('filtered_mixed_coefs_district_square_required.json', 'r') as infile:
    c = pickle.load(infile)

## Save data

In [ ]:
with open('metrovka_flats_metro.json', 'w') as outfile:
    json.dump(metrovka_data, outfile)
#json.dumps(data_2[0:1])